# Formatear salida utilizando foreachRDD

En esta demo se muestra cómo definir la salida de nuestro sistema Spark Streaming.

### Demo

In [ ]:
import findspark
#Importante: Modificar la ruta para que apunte al HOME de Spark
findspark.init('/opt/spark')

In [ ]:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext

### Introducción de datos
Se va a utilizar un socket para introducir datos. Dicho socket estará escuchando en el puerto 9999 de localhost. Para la introducción de datos se puede abrir una ventana de comandos y ejecutar el comando "nc -lk 9999", que abre el puerto y lo mantiene abierto, para a continuación pegar los mensajes que se quieren enviar. Si no existe el comando, deberemos instalar el programa "netcat" --> "sudo apt-get install netcat"

In [ ]:
sc = SparkContext("local[2]", "CalculateSumSquares")
ssc = StreamingContext(sc, 1)

lines = ssc.socketTextStream('localhost',9999)


Se reciben líneas con números separados por espacios, y se calcula la suma de los cuadrados de todos los números recibidos en un mismo batch.

Mediante la función "printAndSave" definimos las acciones a realizar para cada RDD del DStream: por un lado, se imprime por pantalla el RDD tras hacer collect(), y por otro se escribe el contenido del RDD al final de un fichero que se indica por parámetro.

In [ ]:
def sum_of_squares(lines):
    
    nbrs = lines.flatMap(lambda line: line.strip().split(" "))
    result = nbrs.map(lambda x:int(x)**2).reduce(lambda x,y,:x+y)
    return result

def printAndSave(rdd, textFile):
    rdd_c = rdd.collect()
    print(rdd_c)
    with open(textFile, "a+") as outfile:
        outfile.write(str(rdd_c[0]) + "\n")

result = sum_of_squares(lines)
result.foreachRDD(lambda x: printAndSave(x, "./output_test.txt"))


In [ ]:
ssc.start()
# ssc.awaitTermination()

In [ ]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)